In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd

# Device
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS backend for Apple GPU acceleration!")
else:
    device = torch.device("cpu")
    print("MPS not available. Using CPU instead.")

Using MPS backend for Apple GPU acceleration!


In [2]:
class BaselinePINN(nn.Module):
    def __init__(self, hidden_layers=6, hidden_units=512):
        super(BaselinePINN, self).__init__()
        # First layer: input dimension = 2 (x, t)
        layers = []
        in_dim = 2
        
        # Build hidden layers
        for _ in range(hidden_layers):
            layers.append(nn.Linear(in_dim, hidden_units))
            in_dim = hidden_units
        
        # Final output layer: 2 outputs => (psi_real, psi_imag)
        out_dim = 2
        self.layers = nn.ModuleList(layers)
        self.output_layer = nn.Linear(in_dim, out_dim)
        
    def forward(self, inputs):
        """
        x, t: Tensors of shape [batch_size] each,
              representing the coordinates.
        Returns (psi_real, psi_imag) each [batch_size].
        """
        # Combine x,t into [batch_size, 2]
        x, t = inputs
        X = torch.stack((x, t), dim=1)
        
        # Pass through hidden layers with tanh activation
        for layer in self.layers:
            X = layer(X)
            X = torch.tanh(X)
        
        # Final linear output
        out = self.output_layer(X)
        psi_real = out[:, 0]
        psi_imag = out[:, 1]
        return psi_real, psi_imag

In [3]:
# Problem bounds
x_min, x_max = -np.pi, np.pi
t_min, t_max = 0.0, 2.0 * np.pi

n_collocation = 3140
n_boundary = 200
n_initial = 314

def sample_points():
    # Collocation (interior)
    x_f = np.random.uniform(x_min, x_max, n_collocation)
    t_f = np.random.uniform(t_min, t_max, n_collocation)
    
    # Boundary: x=±π, random t
    t_b = np.random.uniform(t_min, t_max, n_boundary // 2)
    x_b_left  = np.full(n_boundary // 2, x_min)
    x_b_right = np.full(n_boundary // 2, x_max)
    x_b = np.concatenate([x_b_left, x_b_right], axis=0)
    t_b = np.concatenate([t_b, t_b], axis=0)
    
    # Initial: t=0, random x in [-π, π]
    x_i = np.random.uniform(x_min, x_max, n_initial)
    t_i = np.zeros(n_initial)

    # Convert to PyTorch
    x_f = torch.from_numpy(x_f).float().to(device)
    t_f = torch.from_numpy(t_f).float().to(device)

    x_b = torch.from_numpy(x_b).float().to(device)
    t_b = torch.from_numpy(t_b).float().to(device)

    x_i = torch.from_numpy(x_i).float().to(device)
    t_i = torch.from_numpy(t_i).float().to(device)

    return x_f, t_f, x_b, t_b, x_i, t_i


In [4]:
def loss_function(model,
                  x_colloc, t_colloc,
                  x_bound, t_bound,
                  x_init, t_init):
    """
    Computes three losses for a 1D harmonic oscillator Schrödinger PINN in real-imag form:
      1) PDE loss: 
         du/dt + 0.5 d²v/dx² - 0.5 x² v = 0
         -dv/dt + 0.5 d²u/dx² - 0.5 x² u = 0
      2) Initial condition loss (u(x,0)=ψ0(x), v(x,0)=0)
      3) Boundary condition loss (u(±π,t)=0, v(±π,t)=0)

    model((x, t)) -> (u, v) = (real part, imaginary part).
    """

    #
    # 1) PDE LOSS
    #
    # Make sure we can take derivatives wrt x_colloc, t_colloc
    x_colloc_ = x_colloc.clone().requires_grad_(True)
    t_colloc_ = t_colloc.clone().requires_grad_(True)
    
    # Evaluate model => u, v
    u, v = model((x_colloc_, t_colloc_))  # each shape [n_colloc]

    # PDE needs partial derivatives
    # du/dt, dv/dt, d²u/dx², d²v/dx²
    du_dt = torch.autograd.grad(
        u, t_colloc_,
        grad_outputs=torch.ones_like(u),
        create_graph=True
    )[0]
    dv_dt = torch.autograd.grad(
        v, t_colloc_,
        grad_outputs=torch.ones_like(v),
        create_graph=True
    )[0]

    du_dx = torch.autograd.grad(
        u, x_colloc_,
        grad_outputs=torch.ones_like(u),
        create_graph=True
    )[0]
    dv_dx = torch.autograd.grad(
        v, x_colloc_,
        grad_outputs=torch.ones_like(v),
        create_graph=True
    )[0]

    d2u_dx2 = torch.autograd.grad(
        du_dx, x_colloc_,
        grad_outputs=torch.ones_like(du_dx),
        create_graph=True
    )[0]
    d2v_dx2 = torch.autograd.grad(
        dv_dx, x_colloc_,
        grad_outputs=torch.ones_like(dv_dx),
        create_graph=True
    )[0]

    # PDE #1: du/dt + 0.5 d²v/dx² - 0.5 x² v = 0
    # PDE #2: -dv/dt + 0.5 d²u/dx² - 0.5 x² u = 0
    # We'll form residual1, residual2, then sum squares
    term1 = du_dt + 0.5 * d2v_dx2 - 0.5*(x_colloc_**2)*v
    term2 = - dv_dt + 0.5 * d2u_dx2 - 0.5*(x_colloc_**2)*u

    pde_loss = torch.mean(term1**2 + term2**2)


    #
    # 2) INITIAL CONDITION LOSS
    #
    # Evaluate model at t=0
    u_i, v_i = model((x_init, t_init))

    # The exact initial wavefunction is purely real => v=0
    # phi0(x) = π^{-1/4} exp(-x²/2)
    # phi1(x) = sqrt(2) x π^{-1/4} exp(-x²/2)
    # => superposition (φ0 + φ1)/√2
    phi0 = (1/(torch.tensor(torch.pi)**0.25)) * torch.exp(-0.5*(x_init**2))
    phi1 = phi0 * (torch.sqrt(torch.tensor(2.0))*x_init)
    psi_init_exact = (phi0 + phi1)/torch.sqrt(torch.tensor(2.0))

    # We want u(x,0) = psi_init_exact, v(x,0)=0
    init_loss = torch.mean((u_i - psi_init_exact)**2) \
              + torch.mean((v_i - 0.0)**2)


    #
    # 3) BOUNDARY CONDITION LOSS
    #
    # Evaluate model at x=±π, for random t
    u_b, v_b = model((x_bound, t_bound))
    # Dirichlet BC => u=0, v=0 on boundary
    boundary_loss = torch.mean(u_b**2) + torch.mean(v_b**2)


    #
    # Return them all
    #
    return pde_loss, init_loss, boundary_loss


In [5]:
# Instantiate the network
model = BaselinePINN(hidden_layers=6, hidden_units=512).to(device)

# Adam optimizer
optimizer = optim.Adam(
    model.parameters(),
    lr=0.001,
    betas=(0.09, 0.999)
)

# Learning rate scheduler: replicate α_t = α0 * 0.9^(t/2000)
def lr_lambda(step):
    return 0.9 ** (step / 2000.0)

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

# Sample points once (baseline approach). If you want to re-sample each iteration, do inside loop
x_f, t_f, x_b, t_b, x_i, t_i = sample_points()

# Training loop
num_iterations = 30000
print_interval = 2000
history = []

for step in range(1, num_iterations+1):
    optimizer.zero_grad()
    
    physics_loss, init_loss, bc_loss = loss_function(model, x_f, t_f, x_b, t_b, x_i, t_i)
    total_loss = physics_loss + init_loss + bc_loss
    
    total_loss.backward()
    optimizer.step()
    scheduler.step()  # apply LR decay
    
    # Logging
    if step % print_interval == 0:
        lr_current = scheduler.get_last_lr()[0]
        print(f"Step={step}, LR={lr_current:.2e}, "
              f"total={total_loss.item():.3e}, "
              f"pde={physics_loss.item():.3e}, "
              f"init={init_loss.item():.3e}, "
              f"bc={bc_loss.item():.3e}")
    
    history.append({
        "step": step,
        "lr": scheduler.get_last_lr()[0],
        "total_loss": total_loss.item(),
        "physics_loss": physics_loss.item(),
        "init_loss": init_loss.item(),
        "bc_loss": bc_loss.item()
    })


Step=2000, LR=9.00e-04, total=3.960e-02, pde=2.687e-02, init=8.713e-03, bc=4.013e-03
Step=4000, LR=8.10e-04, total=1.867e-02, pde=1.430e-02, init=2.246e-03, bc=2.127e-03
Step=6000, LR=7.29e-04, total=1.287e-02, pde=1.068e-02, init=9.628e-04, bc=1.223e-03
Step=8000, LR=6.56e-04, total=4.415e-02, pde=2.247e-02, init=2.099e-02, bc=6.884e-04
Step=10000, LR=5.90e-04, total=9.177e-02, pde=6.494e-02, init=1.696e-02, bc=9.873e-03
Step=12000, LR=5.31e-04, total=7.633e-02, pde=5.757e-02, init=1.022e-02, bc=8.542e-03
Step=14000, LR=4.78e-04, total=7.270e-02, pde=5.273e-02, init=1.103e-02, bc=8.936e-03
Step=16000, LR=4.30e-04, total=3.433e-02, pde=2.546e-02, init=5.047e-03, bc=3.824e-03
Step=18000, LR=3.87e-04, total=6.406e-02, pde=4.603e-02, init=1.010e-02, bc=7.923e-03
Step=20000, LR=3.49e-04, total=4.858e-02, pde=3.416e-02, init=7.519e-03, bc=6.895e-03
Step=22000, LR=3.14e-04, total=6.918e-02, pde=5.008e-02, init=1.038e-02, bc=8.718e-03
Step=24000, LR=2.82e-04, total=9.466e-02, pde=6.663e-02, i

In [7]:
Nx, Nt = 628, 628
x_eval = torch.linspace(x_min, x_max, Nx).to(device)
t_eval = torch.linspace(t_min, t_max, Nt).to(device)
Xg, Tg = torch.meshgrid(x_eval, t_eval, indexing='ij')  # shape [Nx, Nt]

# Flatten
X_flat = Xg.reshape(-1)
T_flat = Tg.reshape(-1)

# Evaluate model
psi_r_pred, psi_i_pred = model((X_flat, T_flat))
psi_r_grid = psi_r_pred.view(Nx, Nt).detach().cpu().numpy()
psi_i_grid = psi_i_pred.view(Nx, Nt).detach().cpu().numpy()

# Probability density
psi_abs2 = psi_r_grid**2 + psi_i_grid**2
